### 3.12 时间换算

时间间隔

In [1]:
from datetime import timedelta
a = timedelta(days=2, hours=6)
b = timedelta(hours=4.5)
c = a + b
print(c.days)
print(c.seconds)
print(c.seconds / 3600)
print(c.total_seconds() / 3600)

2
37800
10.5
58.5


特定的日期时间

In [2]:
from datetime import datetime
a = datetime(2012, 9, 23)
print(a + timedelta(days=10))

2012-10-03 00:00:00


In [3]:
b = datetime(2012, 12, 21)
d = b - a
print(d.days)

89


In [5]:
now = datetime.today()
print(now)

2019-09-17 14:36:08.040070


如果需要处理更为复杂的日期问题，可以试试dateutil模块。  
如dateutil.relativedelta的relativedelta()函数。

### 3.13 计算上周5的日期

要求一周中上一次出现某天的日期，一个优雅、通用的解决方案看起来是这样的。

In [8]:
from datetime import datetime, timedelta

weekdays = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 
           'Friday', 'Saturday', 'Sunday']

def get_prevoius_byday(dayname, start_date=None):
    if start_date is None:
        start_date = datetime.today()
    day_num = start_date.weekday()
    day_num_target = weekdays.index(dayname)
    days_ago = (7 + day_num - day_num_target) % 7
    if days_ago == 0:
        days_ago = 7
    target_date = start_date - timedelta(days = days_ago)
    return target_date

In [9]:
get_prevoius_byday('Saturday')

datetime.datetime(2019, 9, 14, 15, 51, 11, 394590)

用dateutil包解决上述问题。

In [10]:
from datetime import datetime
from dateutil.relativedelta import relativedelta
from dateutil.rrule import *
d = datetime.now()
print(d)

2019-09-17 15:54:42.097078


In [11]:
# Next Friday
print(d + relativedelta(weekday=FR))

2019-09-20 15:54:42.097078


In [12]:
# Last Friday
print(d + relativedelta(weekday=FR(-1)))

2019-09-13 15:54:42.097078


### 3.14 找出当月的日期范围

对日期进行迭代循环并不需要事先构建一个包含所有日期的列表。  
只需要计算范围的开始和结束日期，然后在迭代时利用datetime.timedelta对象来递增日期就可以了。

In [13]:
from datetime import datetime, date, timedelta
import calendar

def get_month_range(start_date=None):
    if start_date is None:
        start_date = date.today().replace(day=1)
    _, days_in_month = calendar.monthrange(start_date.year, start_date.month)
    end_date = start_date + timedelta(days=days_in_month)
    return (start_date, end_date)

In [20]:
a_day = timedelta(days=1)
first_day, last_day = get_month_range()
while first_day < last_day:
    print(first_day, end='\t')
    first_day += a_day

2019-09-01	2019-09-02	2019-09-03	2019-09-04	2019-09-05	2019-09-06	2019-09-07	2019-09-08	2019-09-09	2019-09-10	2019-09-11	2019-09-12	2019-09-13	2019-09-14	2019-09-15	2019-09-16	2019-09-17	2019-09-18	2019-09-19	2019-09-20	2019-09-21	2019-09-22	2019-09-23	2019-09-24	2019-09-25	2019-09-26	2019-09-27	2019-09-28	2019-09-29	2019-09-30	

最理想的方法时创建一个专门处理日期的函数，用生成器很容易实现。

In [22]:
def date_range(start, stop, step):
    while start < stop:
        yield start
        start += step
        
for d in date_range(datetime(2019,9,17), datetime(2019,9,20), timedelta(hours=6)):
    print(d, end='\t')

2019-09-17 00:00:00	2019-09-17 06:00:00	2019-09-17 12:00:00	2019-09-17 18:00:00	2019-09-18 00:00:00	2019-09-18 06:00:00	2019-09-18 12:00:00	2019-09-18 18:00:00	2019-09-19 00:00:00	2019-09-19 06:00:00	2019-09-19 12:00:00	2019-09-19 18:00:00	

### 3.15 将字符串转换为日期

In [23]:
from datetime import datetime
text = '2019-09-17'
y = datetime.strptime(text, '%Y-%m-%d')
z = datetime.now()
print(z - y)

16:16:06.732701


strptime()的性能通常比我们想象的要糟糕很多。  
因为该函数时纯Python代码实现，而且需要处理各种个样的系统区域设定。  
如果要在代码中解析大量的日期，并且事先知道日期的准确格式，  
那么自行实现一个解决方案会获得巨大的性能提升。

In [24]:
# 假设日期是以"YYYY-MM-DD"形式表示的
from datetime import datetime
def  parse_ymd(s):
    year_s, mon_s, day_s = s.split('-')
    return datetime(int(year_s), int(mon_s), int(day_s))

### 3.16 处理涉及到时区的日期问题

In [25]:
from datetime import datetime
from pytz import timezone
d = datetime(2019, 9, 17, 16, 30, 0)
print(d)

2019-09-17 16:30:00


In [27]:
# Localize the date for Chicago（芝加哥）
central = timezone('US/Central')
loc_d = central.localize(d)
print(loc_d)

2019-09-17 16:30:00-05:00


通常用来处理本地时间的方法是将所有的日期都转换为UTC(世界统一时间)时间，   
然后在所有的内部存储和处理中都是用UTC时间。

In [29]:
import pytz
utc_d = loc_d.astimezone(pytz.utc)
print(utc_d)

2019-09-17 21:30:00+00:00


在同时区打交道时，一个常见问题时如何知道时区名称。

In [30]:
# 印度时区
print(pytz.country_timezones['IN'])

['Asia/Kolkata']
